# Crawler de YAPO Inmuebles

### Importar librerias

In [47]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from sklearn.utils import shuffle


### Extraer páginas y url de cada publicación

In [48]:
def get_soup(page ,modalidad="comprar", region = "region_metropolitana"):
    url = f"https://www.yapo.cl/{region}/{modalidad}?ca=15_s&cg=1220&o={page}"
    req = requests.get(url)
    soup = BeautifulSoup(req.text)
    return soup

In [49]:
def get_posts_urls(s):
    posts = s.find_all("tr",{'class':"ad listing_thumbs"})
    posts_urls = [post.find('a',{'class':'redirect-to-url'}).get('href') for post in posts]
    return posts_urls
def get_reqs(s):
    return [requests.get(url) for url in get_posts_urls(s)]


In [50]:
def is_clp(X):
    return "$" in X
def is_uf(X):
    return "UF" in X
def clean_clp(X):
    return X.replace("\n","").replace("\t","").replace("$","")
def clean_uf(X):
    return X.replace("\n","").replace("\t","").replace("UF","")

def clean_price(X):
    if is_clp(X):
        return clean_clp(X)
    elif is_uf(X):
        return clean_uf(X)

def prices_clp_uf(X, Y):
    clp_p , uf_p = str() , str()
    if is_clp(X):
        clp_p = clean_price(X)
        uf_p = clean_price(Y)
    else:
        uf_p = clean_price(X)
        clp_p = clean_price(Y)
    
    return clp_p, uf_p

### Obtener data desde la página de una publicación

In [51]:
def crawl_req(req, modalidad):
    time.sleep(0.1)
    soup_inner = BeautifulSoup(req.text)
    DESCR = soup_inner.find("p",{'itemprop':"description"}).text.replace("\n"," ").strip()
    DETAILS = soup_inner.find("div",{"class":"details"}).find("table")
    Features, Values = DETAILS.findChildren(['th']), DETAILS.findChildren('td')
    
    dic_details = {}
    
    for f, v in zip(Features, Values):
        dic_details[f.text]=v.text.strip()
    
    price = soup_inner.find("div",{"itemprop":"price"}).text.strip()
    price_ref = soup_inner.find("div",{"class":"referencial-price"}).text.replace("(*)","").replace(" ","")
    
    
    price_clp , price_uf = prices_clp_uf(price, price_ref)
    
    data = {}
    data['Tipo de inmueble'] = dic_details['Tipo de inmueble']
    data['Modalidad'] = modalidad
    data['Precio UF'] = price_uf
    data['Precio CLP'] = price_clp
    data['Comuna'] = dic_details['Comuna']
    data['Baños'] = dic_details['Baños'].split(" ")[0]
    data['Dormitorios'] = dic_details['Dormitorios'].split(" ")[0]
    data['Descripcion'] = DESCR
    return data

### Función crawl_yapo_propiedades

Recibe la modalidad de publicación (arrendar, comprar), rangos de páginas a leer, nombre del archivo donde guardar los datos y el paso, con el cual saltar de páginas.

In [61]:
def crawl_yapo_propiedades(modalidad, pages_range, filename = 'inmuebles_yapocl.csv', step=1):
    list_dic = []
    for i in range(pages_range[0],pages_range[1]+1,step):
        print(i)
        s = get_soup(i, modalidad = modalidad)
        reqs = get_reqs(s)
        for req in reqs:
            try:
                list_dic.append(crawl_req(req, modalidad))
            except Exception as e:
                print(e)
    df = pd.DataFrame(list_dic)
    df.to_csv(filename)
    return df


# Ejemplo

In [56]:
%%capture
crawl_sample = crawl_yapo_propiedades("comprar",[1,3])

In [55]:
crawl_sample

,Tipo de inmueble,Modalidad,Precio UF,Precio CLP,Comuna,Baños,Dormitorios,Descripcion
0,Departamento,comprar,"2.463,46",69.000.000,San Miguel,3,2,"Departamentl 2 dormitorio, 2 baños, wolkin clo..."
1,Departamento,comprar,"2.675,00",74.924.957,Santiago,1,2,"2 dormitorio, 1 baño, cocina y logia aparte, l..."
2,Departamento,comprar,"2.106,44",59.000.000,Las Condes,2,3,"Vendo depto 3 dorm, dos baños ubicasomen pasaj..."
3,Casa,comprar,"11.067,74",310.000.000,Ñuñoa,3,4,Se vende linda y amplia casa en excelente esta...
4,Departamento,comprar,"1.695,86",47.500.000,Santiago,1,1,"Vendemos departamento de 2 ambientes, atención..."
5,Casa,comprar,"2.678,00",75.008.985,Peñaflor,2,3,Se vende hermosa casa esquina Av los Rosales e...
6,Casa,comprar,"2.784,78",78.000.000,San Bernardo,4,4,Se vende casa de 2 pisos con 5 dormitorios 2 b...
7,Departamento,comprar,"4.712,71",132.000.000,Ñuñoa,2,3,"Departamento ubicado en Ñuñoa, con excelente c..."
8,Casa,comprar,"3.213,21",90.000.000,Quilicura,3,4,"Hermosa Y AMPLIA casa en ALTOS DE QUILICURA, c..."
9,Casa,comprar,"3.213,21",90.000.000,La Florida,2,4,"Vendo casa 5 dormitorios , Dos baños , buena p..."
